In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from pinecone import Pinecone

# ✅ 반드시 필요 (이 셀에서 .env 로드)
load_dotenv(override=True)

# === 환경변수 읽기 ===
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = "new-realestate"

# === 환경변수 검증 (중요!) ===
if not OPENAI_API_KEY:
    raise ValueError("❌ OPENAI_API_KEY가 설정되지 않았습니다.")
if not PINECONE_API_KEY:
    raise ValueError("❌ PINECONE_API_KEY가 설정되지 않았습니다.")
if not PINECONE_INDEX_NAME:
    raise ValueError("❌ PINECONE_INDEX_NAME가 설정되지 않았습니다 (.env 확인).")

# === 클라이언트 생성 ===
client = OpenAI(api_key=OPENAI_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)

# === 모델 설정 ===
EMBED_MODEL = "text-embedding-3-large"   # 3072-dim
CHAT_MODEL = "gpt-4.1-mini"

print("✅ OpenAI & Pinecone 연결 완료")
print("   - Index:", PINECONE_INDEX_NAME)

✅ OpenAI & Pinecone 연결 완료
   - Index: new-realestate


In [ ]:
import os
from pinecone import Pinecone

# =========================
# CONFIG (여기만 수정)
# =========================
INDEX_NAME = "new-realestate"
NAMESPACE  = "__default__"
DRY_RUN    = False   # True면 삭제 안 함

# =========================
def confirm(msg: str):
    print(msg)
    ans = input("정말 진행할까요? (yes/no): ").strip().lower()
    if ans != "yes":
        raise SystemExit("❌ 사용자 취소")

api_key = os.getenv("PINECONE_API_KEY")
if not api_key:
    raise ValueError("❌ PINECONE_API_KEY 없음")

pc = Pinecone(api_key=api_key)
index = pc.Index(INDEX_NAME)

print("✅ connected to index:", INDEX_NAME)

# =========================
# 🔥 namespace 전체 삭제
# =========================
print(f"🗑️ namespace 전체 삭제: {NAMESPACE}")

if DRY_RUN:
    print("⚠️ DRY_RUN=True → 실제 삭제 안 함")
else:
    confirm(f"⚠️ namespace '{NAMESPACE}'의 모든 vector를 삭제합니다.")
    index.delete(delete_all=True, namespace=NAMESPACE)
    print("✅ namespace 전체 삭제 완료")

✅ connected to index: new-realestate
🗑️ namespace 전체 삭제: __default__
⚠️ namespace '__default__'의 모든 vector를 삭제합니다.


== 파일 변환 ==

In [2]:
# =========================
# PDF / DOCX -> JSONL
# (Jupyter single-cell version)
# =========================

import os
import re
import json
import hashlib
from typing import List, Dict, Any

from pypdf import PdfReader
from docx import Document


# -------------------------
# 1) Text utils
# -------------------------
def clean_text(s: str) -> str:
    if not s:
        return ""
    s = s.replace("\u00a0", " ").replace("\r", "\n")
    s = re.sub(r"[ \t]+", " ", s)
    s = re.sub(r"\n{3,}", "\n\n", s)
    return s.strip()


def chunk_text(text: str, chunk_size: int = 900, overlap: int = 150) -> List[str]:
    text = clean_text(text)
    if not text:
        return []

    chunks = []
    n = len(text)
    start = 0

    while start < n:
        end = min(start + chunk_size, n)
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)

        if end >= n:
            break
        start = max(0, end - overlap)

    return chunks


# -------------------------
# 2) PDF loader (page-based)
# -------------------------
def load_pdf_units(path: str) -> List[Dict[str, Any]]:
    reader = PdfReader(path)
    source = os.path.basename(path)
    units = []

    for i, page in enumerate(reader.pages):
        text = clean_text(page.extract_text() or "")
        if not text:
            continue

        page_no = i + 1
        units.append({
            "source": source,
            "unit_type": "page",
            "unit_key": f"p{page_no}",
            "text": text,
            "meta": {"page": page_no},
        })

    return units


# -------------------------
# 3) DOCX loader (article-based)
# -------------------------
ARTICLE_RE = re.compile(r"(제\s*\d+\s*조(?:의\s*\d+)?)")

def split_korean_law_articles(full_text: str) -> List[Dict[str, Any]]:
    text = clean_text(full_text)
    if not text:
        return []

    matches = list(ARTICLE_RE.finditer(text))
    if not matches:
        return [{"article": None, "article_no": None, "text": text}]

    blocks = []
    for i, m in enumerate(matches):
        start = m.start()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(text)

        article = re.sub(r"\s+", "", m.group(1))
        nums = re.findall(r"\d+", article)
        article_no = int(nums[0]) if nums else None

        blocks.append({
            "article": article,
            "article_no": article_no,
            "text": text[start:end].strip()
        })

    return blocks


def load_docx_units(path: str) -> List[Dict[str, Any]]:
    doc = Document(path)
    paras = [p.text.strip() for p in doc.paragraphs if p.text and p.text.strip()]
    full_text = clean_text("\n".join(paras))
    if not full_text:
        return []

    source = os.path.basename(path)
    units = []

    for i, blk in enumerate(split_korean_law_articles(full_text)):
        unit_key = f"a{blk['article_no']}" if blk["article_no"] else f"blk{i}"
        units.append({
            "source": source,
            "unit_type": "article",
            "unit_key": unit_key,
            "text": blk["text"],
            "meta": {
                "article": blk["article"],
                "article_no": blk["article_no"],
            }
        })

    return units


# -------------------------
# 4) ID / version helpers
# -------------------------
def guess_doc_version(filename: str) -> str:
    m = re.search(r"\((\d{8})\)", filename)
    if m:
        return m.group(1)

    m = re.search(r"[_-](\d{6})", filename)
    if m:
        return m.group(1)

    m = re.search(r"(20\d{2})", filename)
    if m:
        return m.group(1)

    return "unknown"


def make_doc_id(source: str) -> str:
    return hashlib.sha1(source.encode("utf-8")).hexdigest()[:16]


def make_chunk_id(doc_id: str, doc_version: str, unit_type: str, unit_key: str, chunk_id: int) -> str:
    raw = f"{doc_id}|{doc_version}|{unit_type}|{unit_key}|c{chunk_id}"
    return hashlib.sha1(raw.encode("utf-8")).hexdigest()


# -------------------------
# 5) Build JSONL
# -------------------------
def convert_to_jsonl(
    pdf_paths: List[str],
    docx_paths: List[str],
    out_path: str = "out/chunks.jsonl",
    chunk_size: int = 900,
    overlap: int = 150,
):
    os.makedirs(os.path.dirname(out_path), exist_ok=True)

    total_units = 0
    total_chunks = 0

    with open(out_path, "w", encoding="utf-8") as f:

        # ---- PDF (사례집) ----
        for path in pdf_paths:
            source = os.path.basename(path)
            doc_version = guess_doc_version(source)
            doc_id = make_doc_id(source)

            for u in load_pdf_units(path):
                total_units += 1
                chunks = chunk_text(u["text"], chunk_size, overlap)

                for i, ch in enumerate(chunks):
                    rec = {
                        "id": make_chunk_id(doc_id, doc_version, u["unit_type"], u["unit_key"], i),
                        "text": ch,
                        "metadata": {
                            "doc_id": doc_id,
                            "doc_version": doc_version,
                            "category": "case",
                            "doc_type": "casebook",
                            "source": source,
                            "unit_type": u["unit_type"],
                            "unit_key": u["unit_key"],
                            "chunk_id": i,
                            "priority": 60,
                            **u["meta"],
                        }
                    }
                    f.write(json.dumps(rec, ensure_ascii=False) + "\n")
                    total_chunks += 1

        # ---- DOCX (법령) ----
        for path in docx_paths:
            source = os.path.basename(path)
            doc_version = guess_doc_version(source)
            doc_id = make_doc_id(source)
            law_name = source.split("(")[0].strip()

            for u in load_docx_units(path):
                total_units += 1
                chunks = chunk_text(u["text"], chunk_size, overlap)

                for i, ch in enumerate(chunks):
                    rec = {
                        "id": make_chunk_id(doc_id, doc_version, u["unit_type"], u["unit_key"], i),
                        "text": ch,
                        "metadata": {
                            "doc_id": doc_id,
                            "doc_version": doc_version,
                            "category": "law",
                            "doc_type": "law_text",
                            "law_name": law_name,
                            "source": source,
                            "unit_type": u["unit_type"],
                            "unit_key": u["unit_key"],
                            "chunk_id": i,
                            "priority": 95,
                            **u["meta"],
                        }
                    }
                    f.write(json.dumps(rec, ensure_ascii=False) + "\n")
                    total_chunks += 1

    print(f"✅ JSONL 생성 완료: {out_path}")
    print(f"   units={total_units}, chunks={total_chunks}")


# -------------------------
# 6) 실행 부분
# -------------------------
pdfs = [
        "data/raw/case/2025전세피해지원사례집.pdf",
        "data/raw/case/전세피해법률상담사례집_250102.pdf",
    ]

docxs = [
        "data/raw/law/민법(법률)(제20432호)(20260101)-간략.docx",
        "data/raw/law/상가건물 임대차보호법(법률)(제21065호)(20260102).docx",
        "data/raw/law/주택임대차보호법(법률)(제21065호)(20260102).docx",
        "data/raw/rule/주택임대차계약증서의 확정일자 부여 및 정보제공에 관한 규칙(대법원규칙)(제02986호)(20210610).docx",
        "data/raw/rule/주택임대차계약증서상의 확정일자 부여 및 임대차 정보제공에 관한 규칙(법무부령)(제01022호)(20220207).docx",
        "data/raw/rule/주택임대차보호법 시행령(대통령령)(제35947호)(20260102).docx",
    ]

convert_to_jsonl(pdfs, docxs)

✅ JSONL 생성 완료: out/chunks.jsonl
   units=705, chunks=776


In [3]:
# =========================
# JUPYTER: JSONL -> OpenAI Embeddings -> Pinecone Upsert (SAFE)
# - metadata에서 None(null) 제거
# - dict/list 타입도 안전 변환
# =========================

import os, json, time
from typing import Any, Dict, Iterable, List
from openai import OpenAI
from pinecone import Pinecone

# ====== CONFIG (여기만 수정) ======
JSONL_PATH = "out/chunks.jsonl"   # 네 jsonl 경로
INDEX_NAME = "new-realestate"
NAMESPACE  = "__default__"
BATCH_SIZE = 100

EMBED_MODEL = "text-embedding-3-large"  # 3072
# =================================

# ====== Clients ======
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("❌ OPENAI_API_KEY 환경변수 없음")
if not PINECONE_API_KEY:
    raise ValueError("❌ PINECONE_API_KEY 환경변수 없음")

client = OpenAI(api_key=OPENAI_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)

print("✅ clients ready:", INDEX_NAME, NAMESPACE)

# ====== JSONL Loader ======
def load_jsonl(path: str) -> Iterable[Dict[str, Any]]:
    with open(path, "r", encoding="utf-8") as f:
        for line_no, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            try:
                yield json.loads(line)
            except Exception as e:
                raise ValueError(f"❌ JSONL 파싱 실패 (line {line_no}): {e}") from e

# ====== Embedding ======
def embed_texts(texts: List[str], max_retries: int = 5) -> List[List[float]]:
    last_err = None
    for attempt in range(1, max_retries + 1):
        try:
            resp = client.embeddings.create(model=EMBED_MODEL, input=texts)
            return [d.embedding for d in resp.data]
        except Exception as e:
            last_err = e
            wait = min(8.0, 1.5 * attempt)
            print(f"⚠️ embedding retry {attempt}/{max_retries} in {wait:.1f}s: {e}")
            time.sleep(wait)
    raise RuntimeError(f"❌ embedding failed: {last_err}")

# ====== Pinecone Metadata Cleaner (핵심) ======
def clean_value(v: Any) -> Any:
    """
    Pinecone metadata 허용 타입:
    - string, number, boolean
    - list[string]
    None/null 불가
    """
    if v is None:
        return None

    if isinstance(v, (str, int, float, bool)):
        return v

    if isinstance(v, list):
        # list는 string list만 안전. 아니면 문자열로 캐스팅
        return [x if isinstance(x, str) else str(x) for x in v]

    if isinstance(v, dict):
        # dict는 JSON 문자열로 변환
        return json.dumps(v, ensure_ascii=False)

    # 그 외 타입은 문자열로
    return str(v)

def clean_metadata(md: Dict[str, Any]) -> Dict[str, Any]:
    out = {}
    for k, v in md.items():
        vv = clean_value(v)
        if vv is None:
            continue  # ✅ None이면 key 자체 제거
        out[k] = vv
    return out

# ====== Upsert one batch ======
def upsert_batch(rows: List[Dict[str, Any]]):
    # 필수 필드 체크
    for r in rows:
        if "id" not in r or not r["id"]:
            raise ValueError("❌ 어떤 row에 id가 비어있음")
        if "text" not in r or not r["text"]:
            raise ValueError(f"❌ id={r.get('id')} row에 text가 비어있음")

    texts = [r["text"] for r in rows]
    embs = embed_texts(texts)

    vectors = []
    for r, emb in zip(rows, embs):
        md = {
            "article":  r.get("article"),     # ✅ None이면 제거됨
            "chunk_id": r.get("chunk_id"),
            "category": r.get("category"),
            "law_type": r.get("law_type"),
            "priority": r.get("priority"),
            "source":   r.get("source"),
            "text":     r.get("text"),        # RAG 근거 출력용
        }

        vectors.append({
            "id": str(r["id"]),
            "values": emb,
            "metadata": clean_metadata(md),
        })

    index.upsert(vectors=vectors, namespace=NAMESPACE)

# ====== Main loop ======
buffer = []
total = 0
skipped = 0

for row in load_jsonl(JSONL_PATH):
    # row 자체가 이상한 경우 스킵
    if not isinstance(row, dict):
        skipped += 1
        continue
    if not row.get("id") or not row.get("text"):
        skipped += 1
        continue

    buffer.append(row)

    if len(buffer) >= BATCH_SIZE:
        upsert_batch(buffer)
        total += len(buffer)
        print(f"✅ upserted: {total} (skipped: {skipped})")
        buffer.clear()

if buffer:
    upsert_batch(buffer)
    total += len(buffer)

print(f"🎉 DONE. total upserted: {total}, skipped: {skipped}")

✅ clients ready: new-realestate __default__
✅ upserted: 100 (skipped: 0)
✅ upserted: 200 (skipped: 0)
✅ upserted: 300 (skipped: 0)
✅ upserted: 400 (skipped: 0)
✅ upserted: 500 (skipped: 0)
✅ upserted: 600 (skipped: 0)
✅ upserted: 700 (skipped: 0)
🎉 DONE. total upserted: 776, skipped: 0


In [ ]:
# rag_chatbot.py
import os
from dataclasses import dataclass
from typing import Any
from dotenv import load_dotenv
from pinecone import Pinecone
from openai import OpenAI

load_dotenv(override=True)

# ====== ENV ======
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

INDEX_NAME = os.getenv("PINECONE_INDEX_NAME", "new-realestate")
NAMESPACE = os.getenv("PINECONE_NAMESPACE", "__default__")

EMBED_MODEL = os.getenv("EMBED_MODEL", "text-embedding-3-large")  # 3072
CHAT_MODEL = os.getenv("CHAT_MODEL", "gpt-4.1-mini")

TOP_K = int(os.getenv("TOP_K", "6"))
SCORE_THRESHOLD = float(os.getenv("SCORE_THRESHOLD", "0.20"))  # 프로젝트에 맞게 조절

if not OPENAI_API_KEY:
    raise ValueError("❌ OPENAI_API_KEY가 .env에 없습니다.")
if not PINECONE_API_KEY:
    raise ValueError("❌ PINECONE_API_KEY가 .env에 없습니다.")

# ====== CLIENTS ======
client = OpenAI(api_key=OPENAI_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)


@dataclass
class RetrievedChunk:
    id: str
    score: float
    text: str
    metadata: dict[str, Any]


def embed(text: str) -> list[float]:
    r = client.embeddings.create(model=EMBED_MODEL, input=text)
    return r.data[0].embedding


def retrieve(query: str, top_k: int = TOP_K) -> list[RetrievedChunk]:
    qvec = embed(query)

    res = index.query(
        namespace=NAMESPACE,
        vector=qvec,
        top_k=top_k,
        include_metadata=True,
    )

    matches = res.get("matches", []) if isinstance(res, dict) else getattr(res, "matches", [])

    out: list[RetrievedChunk] = []
    for m in matches:
        mid = m.get("id") if isinstance(m, dict) else getattr(m, "id", "")
        score = m.get("score") if isinstance(m, dict) else getattr(m, "score", 0.0)
        md = m.get("metadata", {}) if isinstance(m, dict) else getattr(m, "metadata", {})

        text = (md.get("text") or md.get("chunk") or md.get("content") or "").strip()
        if not text:
            continue

        out.append(RetrievedChunk(id=mid, score=float(score), text=text, metadata=md))

    # score threshold 적용
    out = [c for c in out if c.score >= SCORE_THRESHOLD]
    return out


def build_context(chunks: list[RetrievedChunk]) -> str:
    """
    LLM에게 보여줄 컨텍스트를 깔끔하게 구성.
    메타데이터 키는 너 적재 포맷에 맞춰 유연하게 처리.
    """
    parts = []
    for i, c in enumerate(chunks, 1):
        md = c.metadata or {}
        source = md.get("source", "")
        article = md.get("article", "")
        law_type = md.get("law_type", "")
        chunk_id = md.get("chunk_id", "")

        header_bits = []
        if law_type: header_bits.append(f"law_type={law_type}")
        if source:   header_bits.append(f"source={source}")
        if article:  header_bits.append(f"article={article}")
        if chunk_id != "": header_bits.append(f"chunk_id={chunk_id}")
        header = " | ".join(header_bits) if header_bits else "metadata=unknown"

        parts.append(
            f"[근거 {i}] score={c.score:.4f} id={c.id}\n"
            f"{header}\n"
            f"{c.text}"
        )

    return "\n\n---\n\n".join(parts)


SYSTEM_PROMPT = """\
너는 한국의 전월세(주택임대차) 관련 법률/사례 기반으로 답변하는 어시스턴트다.
반드시 제공된 [근거] 안에서만 사실을 주장하고, 근거가 부족하면 추측하지 말고
제공된 근거로는 판단하기 어렵다"라고 말한다.

임대차에 관한 질문에서는 사용대차(차주)에 관한 규정을 직접적인 법적 근거로 단정적으로 인용하지 않는다.

답변 형식:
1) 결론(한두 문장)
2) 근거 요약(근거 번호 인용해서 bullet)
3) 추가로 확인할 사실(필요하면)
"""

def answer(query: str) -> dict[str, Any]:
    chunks = retrieve(query)
    if not chunks:
        return {
            "answer": "제공된 근거(검색 결과)가 부족해서 답변하기 어렵습니다. 질문을 더 구체화해 주세요.",
            "sources": [],
        }

    context = build_context(chunks)

    msg = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"""\
[질문]
{query}

[근거]
{context}
"""},
    ]

    r = client.chat.completions.create(
        model=CHAT_MODEL,
        messages=msg,
        temperature=0.2,
    )

    text = r.choices[0].message.content

    # 사용자에게 근거 목록도 함께 제공
    sources = []
    for c in chunks:
        md = c.metadata or {}
        sources.append({
            "id": c.id,
            "score": c.score,
            "source": md.get("source"),
            "article": md.get("article"),
            "law_type": md.get("law_type"),
            "chunk_id": md.get("chunk_id"),
        })

    return {"answer": text, "sources": sources}


def main():
    print(f"✅ RAG Chatbot ready (index={INDEX_NAME}, ns={NAMESPACE}, embed={EMBED_MODEL}, chat={CHAT_MODEL})")
    print("종료: Ctrl+C\n")

    while True:
        q = input("Q> ").strip()
        if not q:
            continue

        out = answer(q)
        print("\n=== A ===")
        print(out["answer"])

        print("\n=== SOURCES ===")
        for i, s in enumerate(out["sources"], 1):
            print(f"[{i}] score={s['score']:.4f} law_type={s.get('law_type')} source={s.get('source')} article={s.get('article')} chunk_id={s.get('chunk_id')} id={s.get('id')}")
        print()


if __name__ == "__main__":
    main()

✅ RAG Chatbot ready (index=new-realestate, ns=__default__, embed=text-embedding-3-large, chat=gpt-4.1-mini)
종료: Ctrl+C



In [5]:
# =========================
# (ADD) 카테고리 함수 / 키워드 사전 로딩 + 정규화 유틸
# - 업로드한 `카테고리 함수.txt`, `키워드사전.txt`를 그대로 활용
# =========================
from pathlib import Path
import re

def parse_synonym_rules(raw: str):
    """'A -> B' 형태의 치환 규칙 파싱"""
    rules = []
    for line in raw.splitlines():
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        line = line.strip('",')
        if "->" in line:
            src, dst = [x.strip() for x in line.split("->", 1)]
            if src and dst and src != dst:
                rules.append((src, dst))
    # 긴 표현부터 우선 치환
    rules.sort(key=lambda x: len(x[0]), reverse=True)
    return rules

def normalize_text(text: str, rules):
    out = text
    for src, dst in rules:
        out = out.replace(src, dst)
    return out

# --- 1) 카테고리 함수 로딩 ---
_cat_ns = {}
cat_text = Path("data/카테고리 함수.txt").read_text(encoding="utf-8")
exec(cat_text, _cat_ns)  # get_law_category() 주입
get_law_category = _cat_ns["get_law_category"]
LAW_CATEGORY_WEIGHTS = get_law_category()

# --- 2) 키워드 사전 로딩 ---
_kw_ns = {}
kw_text = Path("data/키워드사전.txt").read_text(encoding="utf-8")
exec(kw_text, _kw_ns)  # keyword_dict 주입
keyword_dict = _kw_ns["keyword_dict"]

SYNONYM_RULES = parse_synonym_rules("\n".join(keyword_dict))

print("✅ loaded:", "LAW_CATEGORY_WEIGHTS=", len(LAW_CATEGORY_WEIGHTS), "synonym_rules=", len(SYNONYM_RULES))


✅ loaded: LAW_CATEGORY_WEIGHTS= 11 synonym_rules= 75


In [6]:
# =========================
# (ADD) 법령/자료 타입 분류 + 중요도(priority) 적용 버전 JSONL 생성
# - 기존 convert_to_jsonl() 코드는 유지
# - priority 규칙:
#   주택임대차보호법=1, 시행령=2, 시행규칙/대법원규칙=3, 민법=4, 사례/판례/피해상담=50
# =========================
import os

PRIORITY_MAP = {
    "주택임대차보호법": 1,
    "시행령": 2,
    "시행규칙/대법원규칙": 3,
    "민법": 4,
    "사례/판례/피해상담": 50,
}

def detect_law_type_from_source(source_filename: str, full_path: str = "") -> str:
    s = source_filename

    # 사례/판례/피해상담: pdf case 폴더 또는 파일명 힌트
    p = (full_path or "").replace("\\", "/")
    if p.endswith(".pdf") or "/case/" in p or "사례" in s or "판례" in s or "상담" in s or "피해" in s:
        return "사례/판례/피해상담"

    # 시행령
    if "시행령" in s:
        return "시행령"

    # 시행규칙/대법원규칙
    if "규칙" in s or "대법원규칙" in s or "법무부령" in s:
        return "시행규칙/대법원규칙"

    # 민법
    if s.startswith("민법") or "민법" in s:
        return "민법"

    # 주택임대차보호법(원문/법률)
    if "주택임대차보호법" in s:
        return "주택임대차보호법"

    # fallback
    return "기타"

def priority_from_law_type(law_type: str) -> int:
    return PRIORITY_MAP.get(law_type, 99)

def convert_to_jsonl_v2(
    pdf_paths,
    docx_paths,
    out_path: str = "out/chunks.jsonl",
    chunk_size: int = 900,
    overlap: int = 150,
):
    """기존 convert_to_jsonl과 동일하게 JSONL을 만들되, law_type/priority를 규칙대로 추가"""
    os.makedirs(os.path.dirname(out_path), exist_ok=True)

    total_units = 0
    total_chunks = 0

    with open(out_path, "w", encoding="utf-8") as f:
        # ---- PDF (사례집 등) ----
        for path in pdf_paths:
            source = os.path.basename(path)
            doc_version = guess_doc_version(source)
            doc_id = make_doc_id(source)

            law_type = detect_law_type_from_source(source, path)
            pr = priority_from_law_type(law_type)

            units = extract_units_from_pdf(path)  # 기존 함수 사용
            for u in units:
                total_units += 1
                chunks = chunk_text(u["text"], chunk_size, overlap)
                for i, ch in enumerate(chunks):
                    rec = {
                        "id": make_chunk_id(doc_id, doc_version, u["unit_type"], u["unit_key"], i),
                        "text": ch,
                        "metadata": {
                            "doc_id": doc_id,
                            "doc_version": doc_version,
                            "category": "law",
                            "doc_type": "case_pdf",
                            "law_type": law_type,
                            "priority": pr,
                            "base_priority": 60,  # 기존 코드와의 호환(참고용)
                            "source": source,
                            "unit_type": u["unit_type"],
                            "unit_key": u["unit_key"],
                            "chunk_id": i,
                            **u["meta"],
                        },
                    }
                    f.write(json.dumps(rec, ensure_ascii=False) + "\n")
                    total_chunks += 1

        # ---- DOCX (법령) ----
        for path in docx_paths:
            source = os.path.basename(path)
            doc_version = guess_doc_version(source)
            doc_id = make_doc_id(source)

            law_name = guess_law_name(source)
            law_type = detect_law_type_from_source(source, path)
            pr = priority_from_law_type(law_type)

            units = extract_units_from_docx(path)  # 기존 함수 사용
            for u in units:
                total_units += 1
                chunks = chunk_text(u["text"], chunk_size, overlap)
                for i, ch in enumerate(chunks):
                    rec = {
                        "id": make_chunk_id(doc_id, doc_version, u["unit_type"], u["unit_key"], i),
                        "text": ch,
                        "metadata": {
                            "doc_id": doc_id,
                            "doc_version": doc_version,
                            "category": "law",
                            "doc_type": "law_text",
                            "law_name": law_name,
                            "law_type": law_type,
                            "priority": pr,
                            "base_priority": 95,  # 기존 코드와의 호환(참고용)
                            "source": source,
                            "unit_type": u["unit_type"],
                            "unit_key": u["unit_key"],
                            "chunk_id": i,
                            **u["meta"],
                        },
                    }
                    f.write(json.dumps(rec, ensure_ascii=False) + "\n")
                    total_chunks += 1

    print(f"✅ JSONL(v2) 생성 완료: {out_path}")
    print(f"   units={total_units}, chunks={total_chunks}")


In [9]:
# =========================
# (ADD) RAG Chatbot v2
# - 사용자 질문: synonym 정규화 적용
# - 카테고리 키워드 가중치로 "재랭킹" (간단 버전)
# - priority(중요도)로 추가 보정
# =========================
from dataclasses import dataclass
from typing import Any, Dict, List, Tuple
import math

@dataclass
class Retrieved:
    id: str
    score: float
    text: str
    metadata: Dict[str, Any]

def embed_query(text: str) -> List[float]:
    # 질문은 정규화 후 임베딩
    norm = normalize_text(text, SYNONYM_RULES)
    r = client.embeddings.create(model=EMBED_MODEL, input=norm)
    return r.data[0].embedding

def keyword_bonus(query_norm: str, chunk_text: str) -> float:
    """LAW_CATEGORY_WEIGHTS 기반: query에 나타난 핵심키워드를 chunk가 포함하면 보너스"""
    bonus = 0.0
    for cat, kwmap in LAW_CATEGORY_WEIGHTS.items():
        # query에 포함된 키워드만 대상으로 축소
        active = [ (k,w) for k,w in kwmap.items() if k and (k in query_norm) ]
        if not active:
            continue
        # chunk에 포함되면 가중치 합산
        hit = 0
        for k,w in active:
            if k in chunk_text:
                hit += w
        if hit:
            # 카테고리별로 완만하게 반영
            bonus += math.log1p(hit) * 0.02
    return bonus

def priority_bonus(md: Dict[str, Any]) -> float:
    p = md.get("priority", 99)
    # 숫자가 작을수록 중요(1~4). case(50)는 보너스 거의 없음
    if p in (1,2,3,4):
        return (5 - p) * 0.02  # 1->0.08, 4->0.02
    return 0.0

def retrieve_v2(question: str, top_k: int = 12, fetch_k: int = 40) -> List[Retrieved]:
    q_norm = normalize_text(question, SYNONYM_RULES)
    q_vec = embed_query(question)

    res = index.query(
        namespace=NAMESPACE,
        vector=q_vec,
        top_k=fetch_k,
        include_metadata=True,
    )

    matches = res.get("matches", []) if isinstance(res, dict) else getattr(res, "matches", [])
    out: List[Retrieved] = []

    for m in matches:
        md = m.get("metadata", {}) if isinstance(m, dict) else getattr(m, "metadata", {})
        text = (md.get("text") or md.get("chunk") or md.get("content") or "").strip()
        if not text:
            continue
        out.append(Retrieved(
            id=m.get("id") if isinstance(m, dict) else getattr(m, "id", ""),
            score=float(m.get("score") if isinstance(m, dict) else getattr(m, "score", 0.0)),
            text=text,
            metadata=md,
        ))

    # --- re-rank ---
    reranked = []
    for r in out:
        b = keyword_bonus(q_norm, r.text) + priority_bonus(r.metadata)
        reranked.append((r.score + b, r))
    reranked.sort(key=lambda x: x[0], reverse=True)

    return [r for _, r in reranked[:top_k]]

def answer_v2(question: str) -> Dict[str, Any]:
    chunks = retrieve_v2(question, top_k=TOP_K, fetch_k=max(30, TOP_K*6))

    # prompt에 "정규화된 질문"도 함께 넣어주면 더 안정적
    q_norm = normalize_text(question, SYNONYM_RULES)

    context = "\n\n".join(
        f"[{i+1}] (law_type={c.metadata.get('law_type')}, priority={c.metadata.get('priority')}, source={c.metadata.get('source')}, unit={c.metadata.get('unit_key')})\n{c.text}"
        for i, c in enumerate(chunks)
    )

    messages = [
        {"role":"system","content":"너는 주택임대차 분야 RAG 챗봇이다. 근거에 기반해 답하고, 모르면 모른다고 말한다."},
        {"role":"user","content":f"""질문: {question}
(정규화 질문: {q_norm})

아래 근거를 참고하여 답변해줘. 근거에 없는 내용은 추측하지 마.

=== 근거 ===
{context}
"""}
    ]

    r = client.chat.completions.create(
        model=CHAT_MODEL,
        messages=messages,
        temperature=0.2,
    )

    ans = r.choices[0].message.content

    sources = []
    for c in chunks:
        md = dict(c.metadata)
        md.update({"id": c.id, "score": c.score})
        sources.append(md)

    return {"answer": ans, "sources": sources}

# --- 간단 실행 ---
def chat_loop_v2():
    while True:
        q = input("\nQ> ").strip()
        if q.lower() in ("q","quit","exit"):
            break
        out = answer_v2(q)
        print("\n=== A ===")
        print(out["answer"])
        print("\n=== SOURCES ===")
        for i, s in enumerate(out["sources"], 1):
            print(f"[{i}] score={s.get('score'):.4f} law_type={s.get('law_type')} priority={s.get('priority')} article={s.get('article')} chunk_id={s.get('chunk_id')} source={s.get('source')}")
